In [1]:
import pandas as pd
from itertools import product

In [2]:
df_original = pd.read_csv("sales_train.csv")
df_calender = pd.read_csv("formatted_calender.csv")

In [3]:
df_original = pd.merge(df_original, df_calender, how="inner", on="date")
df_original

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,Unnamed: 0,day_name,day_of_week,day_of_month,...,month_name,month_of_year,month_year_name,month_year_name_sort,quarter,quarter_year_name_sort,year,year_half,red_day_not_sun,black_friday
0,02.01.2013,0,59,22154,999.0,1.0,1,Wednesday,2,2,...,January,1,January 2013,201301,1,20131,2013,1,1,0
1,02.01.2013,0,25,2565,549.0,1.0,1,Wednesday,2,2,...,January,1,January 2013,201301,1,20131,2013,1,1,0
2,02.01.2013,0,25,2546,299.0,1.0,1,Wednesday,2,2,...,January,1,January 2013,201301,1,20131,2013,1,1,0
3,02.01.2013,0,25,2715,899.0,1.0,1,Wednesday,2,2,...,January,1,January 2013,201301,1,20131,2013,1,1,0
4,02.01.2013,0,25,2716,149.0,1.0,1,Wednesday,2,2,...,January,1,January 2013,201301,1,20131,2013,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,13.10.2015,33,25,7712,149.0,1.0,1015,Tuesday,1,13,...,October,10,October 2015,201510,4,20154,2015,2,0,0
2935845,13.10.2015,33,25,7840,799.0,1.0,1015,Tuesday,1,13,...,October,10,October 2015,201510,4,20154,2015,2,0,0
2935846,13.10.2015,33,25,8113,349.0,1.0,1015,Tuesday,1,13,...,October,10,October 2015,201510,4,20154,2015,2,0,0
2935847,13.10.2015,33,25,8169,248.0,1.0,1015,Tuesday,1,13,...,October,10,October 2015,201510,4,20154,2015,2,0,0


In [4]:
df_draft = df_original[["date_block_num", "shop_id", "item_id", "item_cnt_day", "month_year_name_sort"]].copy()
df_draft = df_draft.groupby(["date_block_num", "item_id", "shop_id", "month_year_name_sort"], as_index=False).sum()
df_result = pd.DataFrame()
for i in range(34):
    df_date_block = df_draft[df_draft["date_block_num"] == i][["date_block_num", "item_id", "shop_id", "month_year_name_sort"]]
    df_crossed = pd.DataFrame(list(product([i], df_date_block["item_id"].unique(), df_date_block["shop_id"].unique(), df_date_block["month_year_name_sort"].unique())),
                               columns=["date_block_num", "item_id", "shop_id", "month_year_name_sort"])
    df_result = pd.concat([df_result, df_crossed], ignore_index=True)
df_result = pd.merge(df_result, df_draft, on=["date_block_num", "item_id", "shop_id", "month_year_name_sort"], how="left")
df_result["item_cnt_day"].fillna(0, inplace=True)
df_result

,date_block_num,item_id,shop_id,month_year_name_sort,item_cnt_day
0,0,19,25,201301,1.0
1,0,19,1,201301,0.0
2,0,19,2,201301,0.0
3,0,19,10,201301,0.0
4,0,19,19,201301,0.0
...,...,...,...,...,...
10913845,33,22167,34,201510,0.0
10913846,33,22167,41,201510,0.0
10913847,33,22167,48,201510,0.0
10913848,33,22167,9,201510,0.0


In [25]:
df_sum = df_result.groupby(["date_block_num", "item_id", "shop_id", "month_year_name_sort"], as_index=False)["item_cnt_day"].sum()

In [26]:
df_lag = df_sum.copy()
df_lag["lag_1"] = df_lag["date_block_num"] + 1
df_lag.isna().any()

date_block_num          False
item_id                 False
shop_id                 False
month_year_name_sort    False
item_cnt_day            False
lag_1                   False
dtype: bool

In [27]:
df_lag = pd.merge(df_sum, df_lag, left_on=["date_block_num", "item_id", "shop_id"], right_on=["lag_1", "item_id", "shop_id"], how="left")
df_lag = df_lag.sort_values("item_cnt_day_x", ascending=False)
df_lag = df_lag.drop(columns=["date_block_num_y", "lag_1", "month_year_name_sort_y"])
df_lag["item_cnt_day_y"].fillna(0, inplace=True)

In [28]:
df_lag = df_lag.rename(columns={"item_cnt_day_x": "item_cnt_month", "item_cnt_day_y": "item_cnt_last_month", "month_year_name_sort_x": "month_year_name_sort"})
df_lag

,date_block_num_x,item_id,shop_id,month_year_name_sort,item_cnt_month,item_cnt_last_month
10794398,33,11373,12,201510,2253.0,246.0
10545641,32,9249,55,201509,1644.0,19.0
10545567,32,9248,12,201509,1644.0,19.0
4471622,11,20949,31,201312,1305.0,989.0
9341405,27,3730,55,201504,1242.0,0.0
...,...,...,...,...,...,...
724919,1,21323,28,201302,-4.0,4.0
6986169,19,11367,12,201408,-5.0,0.0
18549,0,1593,12,201301,-6.0,0.0
18414,0,1590,12,201301,-13.0,0.0


In [29]:
df_lag_12 = df_sum.copy()
df_lag_12["lag_12"] = df_lag_12["date_block_num"]+12
df_lag_12

,date_block_num,item_id,shop_id,month_year_name_sort,item_cnt_day,lag_12
0,0,19,0,201301,0.0,12
1,0,19,1,201301,0.0,12
2,0,19,2,201301,0.0,12
3,0,19,3,201301,0.0,12
4,0,19,4,201301,0.0,12
...,...,...,...,...,...,...
10913845,33,22167,55,201510,0.0,45
10913846,33,22167,56,201510,0.0,45
10913847,33,22167,57,201510,0.0,45
10913848,33,22167,58,201510,0.0,45


In [30]:
df_lag = pd.merge(df_lag, df_lag_12, left_on=["date_block_num_x", "item_id", "shop_id"], right_on=["lag_12", "item_id", "shop_id"], how="left")
df_lag

,date_block_num_x,item_id,shop_id,month_year_name_sort_x,item_cnt_month,item_cnt_last_month,date_block_num,month_year_name_sort_y,item_cnt_day,lag_12
0,33,11373,12,201510,2253.0,246.0,21.0,201410.0,68.0,33.0
1,32,9249,55,201509,1644.0,19.0,NaN,NaN,NaN,NaN
2,32,9248,12,201509,1644.0,19.0,NaN,NaN,NaN,NaN
3,11,20949,31,201312,1305.0,989.0,NaN,NaN,NaN,NaN
4,27,3730,55,201504,1242.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
10913845,1,21323,28,201302,-4.0,4.0,NaN,NaN,NaN,NaN
10913846,19,11367,12,201408,-5.0,0.0,NaN,NaN,NaN,NaN
10913847,0,1593,12,201301,-6.0,0.0,NaN,NaN,NaN,NaN
10913848,0,1590,12,201301,-13.0,0.0,NaN,NaN,NaN,NaN


In [31]:
df_lag = df_lag.drop(columns=["date_block_num", "lag_12", "month_year_name_sort_y"])
df_lag = df_lag.rename(columns={"item_cnt_day": "item_cnt_last_year"})
df_lag


,date_block_num_x,item_id,shop_id,month_year_name_sort_x,item_cnt_month,item_cnt_last_month,item_cnt_last_year
0,33,11373,12,201510,2253.0,246.0,68.0
1,32,9249,55,201509,1644.0,19.0,NaN
2,32,9248,12,201509,1644.0,19.0,NaN
3,11,20949,31,201312,1305.0,989.0,NaN
4,27,3730,55,201504,1242.0,0.0,NaN
...,...,...,...,...,...,...,...
10913845,1,21323,28,201302,-4.0,4.0,NaN
10913846,19,11367,12,201408,-5.0,0.0,NaN
10913847,0,1593,12,201301,-6.0,0.0,NaN
10913848,0,1590,12,201301,-13.0,0.0,NaN


In [32]:
df_lag["item_cnt_last_year"].fillna(0, inplace=True)
df_lag

,date_block_num_x,item_id,shop_id,month_year_name_sort_x,item_cnt_month,item_cnt_last_month,item_cnt_last_year
0,33,11373,12,201510,2253.0,246.0,68.0
1,32,9249,55,201509,1644.0,19.0,0.0
2,32,9248,12,201509,1644.0,19.0,0.0
3,11,20949,31,201312,1305.0,989.0,0.0
4,27,3730,55,201504,1242.0,0.0,0.0
...,...,...,...,...,...,...,...
10913845,1,21323,28,201302,-4.0,4.0,0.0
10913846,19,11367,12,201408,-5.0,0.0,0.0
10913847,0,1593,12,201301,-6.0,0.0,0.0
10913848,0,1590,12,201301,-13.0,0.0,0.0


In [34]:
df_calender_draft = df_calender[["month_year_name_sort", "month_year_name", "month_of_year", "quarter", "year_half", "year"]]

In [42]:
df_lag = df_lag.rename(columns={"month_year_name_sort_x": "month_year_name_sort"})
df_lag

,date_block_num_x,item_id,shop_id,month_year_name_sort,item_cnt_month,item_cnt_last_month,item_cnt_last_year
0,33,11373,12,201510,2253.0,246.0,68.0
1,32,9249,55,201509,1644.0,19.0,0.0
2,32,9248,12,201509,1644.0,19.0,0.0
3,11,20949,31,201312,1305.0,989.0,0.0
4,27,3730,55,201504,1242.0,0.0,0.0
...,...,...,...,...,...,...,...
10913845,1,21323,28,201302,-4.0,4.0,0.0
10913846,19,11367,12,201408,-5.0,0.0,0.0
10913847,0,1593,12,201301,-6.0,0.0,0.0
10913848,0,1590,12,201301,-13.0,0.0,0.0


In [54]:
df_calender_draft_2 = df_calender_draft.groupby(["month_year_name_sort", "month_year_name", 
                                                 "month_of_year", "quarter", "year_half", "year"], as_index=False).count()
df_calender_draft_2

,month_year_name_sort,month_year_name,month_of_year,quarter,year_half,year
0,201301,January 2013,1,1,1,2013
1,201302,February 2013,2,1,1,2013
2,201303,March 2013,3,1,1,2013
3,201304,April 2013,4,2,1,2013
4,201305,May 2013,5,2,1,2013
5,201306,June 2013,6,2,1,2013
6,201307,July 2013,7,3,2,2013
7,201308,August 2013,8,3,2,2013
8,201309,September 2013,9,3,2,2013
9,201310,October 2013,10,4,2,2013


In [55]:
df_final = pd.merge(df_lag, df_calender_draft_2, how="inner", on="month_year_name_sort")
df_final

,date_block_num_x,item_id,shop_id,month_year_name_sort,item_cnt_month,item_cnt_last_month,item_cnt_last_year,month_year_name,month_of_year,quarter,year_half,year
0,33,11373,12,201510,2253.0,246.0,68.0,October 2015,10,4,2,2015
1,33,20949,12,201510,500.0,0.0,0.0,October 2015,10,4,2,2015
2,33,20949,25,201510,461.0,388.0,491.0,October 2015,10,4,2,2015
3,33,20949,42,201510,446.0,387.0,407.0,October 2015,10,4,2,2015
4,33,20949,31,201510,431.0,473.0,634.0,October 2015,10,4,2,2015
...,...,...,...,...,...,...,...,...,...,...,...,...
10913845,1,6664,32,201302,-1.0,0.0,0.0,February 2013,2,1,1,2013
10913846,1,6664,41,201302,-1.0,0.0,0.0,February 2013,2,1,1,2013
10913847,1,19638,53,201302,-1.0,1.0,0.0,February 2013,2,1,1,2013
10913848,1,4691,12,201302,-1.0,1.0,0.0,February 2013,2,1,1,2013


In [59]:
df_work = pd.read_csv("work_days.csv")
df_work= df_work.drop(columns="Unnamed: 0")

,month_year_name,red_day_not_sun
0,April 2013,30
1,April 2014,30
2,April 2015,30
3,August 2013,31
4,August 2014,31
5,August 2015,31
6,December 2013,31
7,December 2014,31
8,February 2013,27
9,February 2014,27


In [60]:
df_final = pd.merge(df_final, df_work, how="inner", on="month_year_name")
df_final

,date_block_num_x,item_id,shop_id,month_year_name_sort,item_cnt_month,item_cnt_last_month,item_cnt_last_year,month_year_name,month_of_year,quarter,year_half,year,red_day_not_sun
0,33,11373,12,201510,2253.0,246.0,68.0,October 2015,10,4,2,2015,31
1,33,20949,12,201510,500.0,0.0,0.0,October 2015,10,4,2,2015,31
2,33,20949,25,201510,461.0,388.0,491.0,October 2015,10,4,2,2015,31
3,33,20949,42,201510,446.0,387.0,407.0,October 2015,10,4,2,2015,31
4,33,20949,31,201510,431.0,473.0,634.0,October 2015,10,4,2,2015,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10913845,1,6664,32,201302,-1.0,0.0,0.0,February 2013,2,1,1,2013,27
10913846,1,6664,41,201302,-1.0,0.0,0.0,February 2013,2,1,1,2013,27
10913847,1,19638,53,201302,-1.0,1.0,0.0,February 2013,2,1,1,2013,27
10913848,1,4691,12,201302,-1.0,1.0,0.0,February 2013,2,1,1,2013,27


In [61]:
df_final.to_csv("sales_train_final.csv")